# Data cleaning

In this file, we cleaned the downloaded data:
The main steps include:
1. Adding party affiliation to tweet rows
2. Deleting unnecessary downloaded Retweets and duplicated tweets.
3. Deleting links and mentions from the tweets text and saving them to separate columns
4. Expanding the column of public metrics
5. Encoding emojis in a unified format
6. Translating tweets using Google Translate in Google Sheets
7. Saving all downloaded tweets to one file

### 1.  Used libraries

In [95]:
import os
import pandas as pd
import pandas as pd
import re
import emoji

### 2. Reading JSON files and transforming them into party-specific pickle files

In [27]:
base_input_paths = ['../data/01.raw/tweets_after_elections', '../data/01.raw/tweets_before_elections']
subfolders = ['Konfederacja', 'NL', 'PIS', 'PO', 'PL2050', 'PSL']
output_folder = '../data/02.processed'

for subfolder in subfolders:
    dataframes = []
    for base_input_path in base_input_paths:
        folder_path = os.path.join(base_input_path, subfolder)
        for filename in os.listdir(folder_path):
            if filename.endswith('.json'):
                file_path = os.path.join(folder_path, filename)
                politician = filename.split("_tweets.json")[0]
                try:
                    df = pd.read_json(file_path)  
                    df["username"] = politician  
                    df["party"] = subfolder
                    print(f"Read {len(df)} rows from {file_path}")  
                    dataframes.append(df)
                except ValueError as e:
                    print(f"Error reading {file_path}: {e}")
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        
        output_file_path = os.path.join(output_folder, f'{subfolder}_combined.pkl')
        combined_df.to_pickle(output_file_path) 
        
        print(f"Saved {subfolder} combined data to {output_file_path}")

print("Processing complete!")

Read 964 rows from ../data/01.raw/tweets_after_elections\Konfederacja\bartlomiejpejo_2023-10-16_2024-10-15.json
Read 889 rows from ../data/01.raw/tweets_after_elections\Konfederacja\GrzegorzBraun__2023-10-16_2024-10-15.json
Read 11 rows from ../data/01.raw/tweets_after_elections\Konfederacja\Iwaszkiewicz_RJ_2023-10-16_2024-10-15.json
Read 289 rows from ../data/01.raw/tweets_after_elections\Konfederacja\KonradBerkowicz_2023-10-15_2024-04-16_vol2 (1).json
Read 1318 rows from ../data/01.raw/tweets_after_elections\Konfederacja\KonradBerkowicz_2024-04-16_2024-10-15_vol1 (1).json
Read 772 rows from ../data/01.raw/tweets_after_elections\Konfederacja\MarSypniewski_2023-10-16_2024-10-15.json
Read 597 rows from ../data/01.raw/tweets_after_elections\Konfederacja\MichalWawer_2023-10-16_2024-10-15.json
Read 421 rows from ../data/01.raw/tweets_after_elections\Konfederacja\placzekgrzegorz_2023-10-16_2024-04-15.json
Read 320 rows from ../data/01.raw/tweets_after_elections\Konfederacja\placzekgrzegorz_

### 3. Data cleaning

In [28]:
df_konfederacja = pd.read_pickle(os.path.join(output_folder, 'Konfederacja_combined.pkl'))
df_NL = pd.read_pickle(os.path.join(output_folder, 'NL_combined.pkl'))
df_PIS = pd.read_pickle(os.path.join(output_folder, 'PIS_combined.pkl'))
df_PO = pd.read_pickle(os.path.join(output_folder, 'PO_combined.pkl'))
df_PL2050 = pd.read_pickle(os.path.join(output_folder, 'PL2050_combined.pkl'))
df_PSL = pd.read_pickle(os.path.join(output_folder, 'PSL_combined.pkl'))

In [29]:
df_konfederacja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12651 entries, 0 to 12650
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   public_metrics          12651 non-null  object             
 1   in_reply_to_user_id     3078 non-null   float64            
 2   reply_settings          12651 non-null  object             
 3   author_id               12651 non-null  float64            
 4   context_annotations     1334 non-null   object             
 5   id                      12651 non-null  float64            
 6   text                    12651 non-null  object             
 7   edit_controls           12651 non-null  object             
 8   referenced_tweets       4543 non-null   object             
 9   created_at              12651 non-null  datetime64[ns, UTC]
 10  edit_history_tweet_ids  12651 non-null  object             
 11  lang                    12651 non-null  o

In [30]:
df_konfederacja.head()

,public_metrics,in_reply_to_user_id,reply_settings,author_id,context_annotations,id,text,edit_controls,referenced_tweets,created_at,edit_history_tweet_ids,lang,conversation_id,entities,possibly_sensitive,category,attachments,geo,username,party
0,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",375146901,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1846277256509116672,"@donaldtusk Niezrealizowanie większości ze ""10...","{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'replied_to', 'id': '184609177626996...",2024-10-15 19:49:34+00:00,[1846277256509116623],pl,1846091776269963776,"{'mentions': [{'start': 0, 'end': 11, 'usernam...",False,Reply,NaN,NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
1,"{'retweet_count': 9, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1846222583898784000,Rok po wyborach trzeba powiedzieć jedno - nie ...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 16:12:19+00:00,[1846222583898784025],pl,1846222583898784000,"{'urls': [{'start': 100, 'end': 123, 'url': 'h...",False,Original,{'media_keys': ['13_1846222491456282626']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
2,"{'retweet_count': 4, 'reply_count': 3, 'like_c...",NaN,everyone,1182211615,NaN,1846161400328028160,"❌Mamy rok po wyborach, a Polska pogrąża się w ...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 12:09:12+00:00,[1846161400328028272],pl,1846161400328028160,"{'urls': [{'start': 278, 'end': 301, 'url': 'h...",False,Original,{'media_keys': ['3_1846148786910810112']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
3,"{'retweet_count': 6, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,NaN,1846091824101769472,Mija rok od wyborów parlamentarnych. W kampani...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 07:32:44+00:00,[1846091824101769490],pl,1846091824101769472,"{'urls': [{'start': 278, 'end': 301, 'url': 'h...",False,Original,{'media_keys': ['3_1846091818959597568']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
4,"{'retweet_count': 45, 'reply_count': 18, 'like...",NaN,everyone,1182211615,NaN,1846075343188144128,#Idę11 🇵🇱 https://t.co/KiCe5ATOpX,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 06:27:14+00:00,[1846075343188144153],qme,1846075343188144128,"{'hashtags': [{'start': 0, 'end': 6, 'tag': 'I...",False,Original,{'media_keys': ['13_1846075276687478784']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja


In [101]:
# Merge all dataframes into one
df = pd.concat([df_konfederacja, df_NL, df_PIS, df_PO, df_PL2050, df_PSL], ignore_index=True)

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52787 entries, 0 to 52786
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   public_metrics          52787 non-null  object             
 1   in_reply_to_user_id     10972 non-null  float64            
 2   reply_settings          52787 non-null  object             
 3   author_id               52787 non-null  float64            
 4   context_annotations     5948 non-null   object             
 5   id                      52771 non-null  float64            
 6   text                    52787 non-null  object             
 7   edit_controls           52771 non-null  object             
 8   referenced_tweets       19691 non-null  object             
 9   created_at              52787 non-null  datetime64[ns, UTC]
 10  edit_history_tweet_ids  52771 non-null  object             
 11  lang                    52787 non-null  o

In [82]:
len(df)

52787

In [83]:
pd.options.display.float_format = '{:.0f}'.format
df['id'] = df['id'].fillna(0).astype('int64')
df['id']

0        1846277256509116672
1        1846222583898784000
2        1846161400328028160
3        1846091824101769472
4        1846075343188144128
                ...         
52782    1701274354145780224
52783    1701273238263742720
52784    1701273238263742720
52785    1697128952131661824
52786    1697128952131661824
Name: id, Length: 52787, dtype: int64

In [86]:
duplicates = df[df.duplicated(subset=['id'], keep=False)]
duplicates

,public_metrics,in_reply_to_user_id,reply_settings,author_id,context_annotations,id,text,edit_controls,referenced_tweets,created_at,edit_history_tweet_ids,lang,conversation_id,entities,possibly_sensitive,category,attachments,geo,username,party
962,"{'retweet_count': 15, 'reply_count': 28, 'like...",NaN,everyone,1182211615,NaN,1714195119706890496,Serdeczne dzięki za każdy głos. 🤝\nDla mnie to...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2023-10-17 08:22:19+00:00,[1714195119706890463],pl,1714195119706890496,"{'hashtags': [{'start': 251, 'end': 264, 'tag'...",False,Original,{'media_keys': ['3_1714195114472431617']},{'place_id': '535f0c2de0121451'},bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
963,"{'retweet_count': 15, 'reply_count': 28, 'like...",NaN,everyone,1182211615,NaN,1714195119706890496,Serdeczne dzięki za każdy głos. 🤝\nDla mnie to...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2023-10-17 08:22:19+00:00,[1714195119706890463],pl,1714195119706890496,"{'urls': [{'start': 275, 'end': 298, 'url': 'h...",False,Original,{'media_keys': ['3_1714195114472431617']},{'place_id': '535f0c2de0121451'},bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
1660,"{'retweet_count': 38, 'reply_count': 23, 'like...",NaN,everyone,1186367483546165248,NaN,1768754497738678272,Skandal👇 https://t.co/XBJRvv3OVR,"{'edits_remaining': 5, 'is_edit_eligible': Tru...","[{'type': 'quoted', 'id': '1768727139937828957'}]",2024-03-15 21:41:48+00:00,[1768754497738678324],in,1768754497738678272,"{'urls': [{'start': 9, 'end': 32, 'url': 'http...",False,Quote,NaN,NaN,GrzegorzBraun__2023-10-16_2024-10-15.json,Konfederacja
1661,"{'retweet_count': 38, 'reply_count': 23, 'like...",NaN,everyone,1186367483546165248,NaN,1768754497738678272,Skandal👇 https://t.co/XBJRvv3OVR,"{'edits_remaining': 5, 'is_edit_eligible': Tru...","[{'type': 'quoted', 'id': '1768727139937828957'}]",2024-03-15 21:41:48+00:00,[1768754497738678324],in,1768754497738678272,"{'urls': [{'start': 9, 'end': 32, 'url': 'http...",False,Quote,NaN,NaN,GrzegorzBraun__2023-10-16_2024-10-15.json,Konfederacja
1743,"{'retweet_count': 16, 'reply_count': 0, 'like_...",NaN,everyone,1186367483546165248,NaN,1753779932742799616,RT @Roman_Korona: ⏰Zapraszam na doroczny Zlot ...,"{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'retweeted', 'id': '1753770538424504...",2024-02-03 13:58:14+00:00,[1753779932742799591],pl,1753779932742799616,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",False,Retweet,NaN,NaN,GrzegorzBraun__2023-10-16_2024-10-15.json,Konfederacja
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52744,"{'retweet_count': 6, 'reply_count': 0, 'like_c...",NaN,everyone,961181894,NaN,1707867087899705600,RT @JKaminska02: Niezależnie od poglądów warto...,"{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'retweeted', 'id': '1707772421258813...",2023-09-29 21:16:59+00:00,[1707867087899705655],pl,1707867087899705600,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",0,Retweet,NaN,NaN,StruzikAdam_2022-10-16_2023-10-15.json,PSL
52783,"{'retweet_count': 41, 'reply_count': 0, 'like_...",NaN,everyone,961181894,NaN,1701273238263742720,RT @KosiniakKamysz: Polska wymiera! Miesięczni...,"{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'retweeted', 'id': '1701264896711709...",2023-09-11 16:35:22+00:00,[1701273238263742680],pl,1701273238263742720,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",0,Retweet,NaN,NaN,StruzikAdam_2022-10-16_2023-10-15.json,PSL
52784,"{'retweet_count': 41, 'reply_count': 0, 'like_...",NaN,everyone,961181894,NaN,1701273238263742720,RT @KosiniakKamysz: Polska wymiera! Miesięczni...,"{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'retweeted', 'id': '1701264896711709...",2023-09-11 16:35:22+00:00,[1701273238263742680],pl,1701273238263742720,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",0,Retweet,NaN,NaN,StruzikAdam_2022-10-16_2023-10-15.json,PSL
52785,"{'retweet_coun

In [ ]:
# Get all duplicate ID rows
duplicate_id_rows = df[df.duplicated(subset=['id'], keep=False)]

# Get all duplicate Text rows
duplicate_text_rows = df[df.duplicated(subset=['text'], keep=False)]

# Get rows where both ID and Text are duplicated
duplicate_id_text_rows = df[df.duplicated(subset=['id', 'text'], keep=False)]

# Compare overlaps
print(f"Rows where ID is duplicated: {len(duplicate_id_rows)}")
print(f"Rows where Text is duplicated: {len(duplicate_text_rows)}")
print(f"Rows where BOTH ID and Text are duplicated: {len(duplicate_id_text_rows)}")

Rows where ID is duplicated: 473
Rows where Text is duplicated: 623
Rows where BOTH ID and Text are duplicated: 457


In [94]:
empty_id_rows = df[df['id'].isna()]
print(f"Rows where `id` is empty (NaN): {len(empty_id_rows)}")
#empty_id_rows

zero_id_rows = df[df['id'] == 0]
print(f"Rows where `id` is 0: {len(zero_id_rows)}")
zero_id_rows

Rows where `id` is empty (NaN): 0
Rows where `id` is 0: 16


,public_metrics,in_reply_to_user_id,reply_settings,author_id,context_annotations,id,text,edit_controls,referenced_tweets,created_at,edit_history_tweet_ids,lang,conversation_id,entities,possibly_sensitive,category,attachments,geo,username,party
34528,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,everyone,61552404,NaN,0,@tomekbit ✌️,NaN,NaN,2023-10-16 00:00:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO
34529,"{'retweet_count': 0, 'reply_count': 2, 'like_c...",NaN,everyone,61552404,NaN,0,"@MaciejGdynia Maćku, czekam na oficjalne wynik...",NaN,NaN,2023-10-16 00:00:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO
34530,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,everyone,61552404,NaN,0,"@MCichonAlicja Alu, czekamy jeszcze na wynik?",NaN,NaN,2023-10-16 00:00:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO
34531,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",NaN,everyone,61552404,NaN,0,@REL_76 🥰🥰🥰,NaN,NaN,2023-10-16 00:00:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO
34532,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",NaN,everyone,61552404,NaN,0,@Gidziela 🥰✌️,NaN,NaN,2023-10-16 00:00:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO
34533,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,everyone,61552404,NaN,0,@WHaptar Gratulacje👏🥂,NaN,NaN,2023-10-16 00:00:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO
34534,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,everyone,61552404,NaN,0,@KapenGenezyp Dziękuję❤️❤️❤️,NaN,NaN,2023-10-16 00:00:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO
34535,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,everyone,61552404,NaN,0,@jasinska_e ❤️,NaN,NaN,2023-10-16 21:57:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO
34536,"{'retweet_count': 18, 'reply_count': 31, 'like...",NaN,everyone,61552404,NaN,0,@BMikolajewska odpowie💪,NaN,NaN,2023-10-16 22:27:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO
34537,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,everyone,61552404,NaN,0,@DorotaNiedziela ja Tobie też❣️,NaN,NaN,2023-10-16 22:41:00+00:00,NaN,pl,NaN,NaN,NaN,Reply,NaN,NaN,Leszczyna_2023-10-16_2023-12-31.json,PO


We need to remove duplicate tweets + delete the tweets that were 0; because our custom downloading loop occasionally downloads the same tweet two times to ensure completeness.

In [102]:
# Copy the original DataFrame before cleaning
df_before = df.copy()

# Get the initial size
initial_size = len(df_before)
print(f"Initial number of tweets: {initial_size}")

# Check and report issues with the 'id' column
print("\n--- ID QUALITY CHECK ---")

# Convert id to string for consistent checking
df_before['id'] = df_before['id'].astype(str)

# Check for various problems
missing_ids = df_before['id'].isna()
empty_ids = df_before['id'] == ''
zero_ids = df_before['id'] == '0'
very_short_ids = df_before['id'].str.len() < 5  # Twitter IDs are typically longer

# Report on ID issues
print(f"Missing IDs (NaN): {missing_ids.sum()} ({missing_ids.mean():.2%})")
print(f"Empty IDs: {empty_ids.sum()} ({empty_ids.mean():.2%})")
print(f"Zero IDs ('0'): {zero_ids.sum()} ({zero_ids.mean():.2%})")
print(f"Very short IDs (< 5 chars): {very_short_ids.sum()} ({very_short_ids.mean():.2%})")

# Create a mask for all problematic IDs
problematic_ids_mask = missing_ids | empty_ids | zero_ids | very_short_ids

# Report total problematic IDs
print(f"Total problematic IDs: {problematic_ids_mask.sum()} ({problematic_ids_mask.mean():.2%})")

# First filter out problematic IDs from the original dataset
df_no_problems = df_before[~problematic_ids_mask].copy()
problematic_removed = initial_size - len(df_no_problems)

# Then remove duplicates from the dataset without problematic IDs
df_after = df_no_problems.drop_duplicates(subset=['id'])
duplicates_removed = len(df_no_problems) - len(df_after)

# Calculate removed counts and percentages
remaining_final = len(df_after)
total_removed = initial_size - remaining_final

duplicate_percentage = (duplicates_removed / initial_size) * 100
problematic_percentage = (problematic_removed / initial_size) * 100
total_removed_percentage = (total_removed / initial_size) * 100
remaining_percentage = (remaining_final / initial_size) * 100

# Print comprehensive results
print("\n--- CLEANING SUMMARY ---")
print(f"Initial tweets: {initial_size}")
print(f"Problematic ID tweets removed: {problematic_removed} ({problematic_percentage:.2f}%)")
print(f"Duplicate tweets removed: {duplicates_removed} ({duplicate_percentage:.2f}%)")
print(f"Total tweets removed: {total_removed} ({total_removed_percentage:.2f}%)")
print(f"Tweets remaining: {remaining_final} ({remaining_percentage:.2f}%)")

# Show sample of problematic IDs
if problematic_ids_mask.sum() > 0:
    print("\nSample of problematic IDs:")
    sample_problematic = df_before[problematic_ids_mask].head(5)
    for i, (idx, row) in enumerate(sample_problematic.iterrows()):
        print(f"  {i+1}. ID: '{row['id']}', Text: '{row['text'][:50]}...'")

# Identify the actual duplicate IDs from the data without problematic IDs
duplicate_ids = df_no_problems[df_no_problems.duplicated(subset=['id'], keep='first')]['id'].unique().tolist()
print(f"\nNumber of unique duplicate IDs: {len(duplicate_ids)}")
if duplicate_ids:
    print("Sample of duplicate IDs (first 5):")
    for i, dup_id in enumerate(duplicate_ids[:5]):
        print(f"  {i+1}. {dup_id}")
else:
    print("No duplicates found")

# Keep df_after as the new df
df = df_after
print(f"\nFinal clean dataframe shape: {df.shape}")

# Verify no problematic IDs remain
if (df['id'] == '0').sum() > 0 or df['id'].isna().sum() > 0 or (df['id'] == '').sum() > 0 or (df['id'].str.len() < 5).sum() > 0:
    print("WARNING: Some problematic IDs still remain in the cleaned dataframe")
else:
    print("SUCCESS: All problematic IDs have been removed")

Initial number of tweets: 52787

--- ID QUALITY CHECK ---
Missing IDs (NaN): 0 (0.00%)
Empty IDs: 0 (0.00%)
Zero IDs ('0'): 0 (0.00%)
Very short IDs (< 5 chars): 16 (0.03%)
Total problematic IDs: 16 (0.03%)

--- CLEANING SUMMARY ---
Initial tweets: 52787
Problematic ID tweets removed: 16 (0.03%)
Duplicate tweets removed: 229 (0.43%)
Total tweets removed: 245 (0.46%)
Tweets remaining: 52542 (99.54%)

Sample of problematic IDs:
  1. ID: 'nan', Text: '@tomekbit ✌️...'
  2. ID: 'nan', Text: '@MaciejGdynia Maćku, czekam na oficjalne wyniki, ż...'
  3. ID: 'nan', Text: '@MCichonAlicja Alu, czekamy jeszcze na wynik?...'
  4. ID: 'nan', Text: '@REL_76 🥰🥰🥰...'
  5. ID: 'nan', Text: '@Gidziela 🥰✌️...'

Number of unique duplicate IDs: 228
Sample of duplicate IDs (first 5):
  1. 1.7141951197068905e+18
  2. 1.7687544977386783e+18
  3. 1.7537799327427996e+18
  4. 1.7367481647223854e+18
  5. 1.7621091238002934e+18

Final clean dataframe shape: (52542, 20)
SUCCESS: All problematic IDs have been remove

In [103]:
# 1) How many total rows have a duplicate 'id' (including the first occurrence)?
total_dup_rows = df.duplicated(subset=['id'], keep=False).sum()
print(f"Total rows that share a duplicate ID (including the first occurrence): {total_dup_rows}")

# 2) How many rows are "extra" duplicates beyond the first?
extra_dup_rows = df.duplicated(subset=['id'], keep='first').sum()
print(f"Number of extra duplicates beyond the first occurrence: {extra_dup_rows}")

# 3) How many unique IDs appear more than once?
duplicate_ids = df[df.duplicated(subset=['id'], keep=False)]['id'].unique()
num_duplicate_ids = len(duplicate_ids)
print(f"Number of unique IDs that are duplicated: {num_duplicate_ids}")

Total rows that share a duplicate ID (including the first occurrence): 0
Number of extra duplicates beyond the first occurrence: 0
Number of unique IDs that are duplicated: 0


In [104]:
df.head()

,public_metrics,in_reply_to_user_id,reply_settings,author_id,context_annotations,id,text,edit_controls,referenced_tweets,created_at,edit_history_tweet_ids,lang,conversation_id,entities,possibly_sensitive,category,attachments,geo,username,party
0,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",375146901,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.8462772565091167e+18,"@donaldtusk Niezrealizowanie większości ze ""10...","{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'replied_to', 'id': '184609177626996...",2024-10-15 19:49:34+00:00,[1846277256509116623],pl,1846091776269963776,"{'mentions': [{'start': 0, 'end': 11, 'usernam...",False,Reply,NaN,NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
1,"{'retweet_count': 9, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.846222583898784e+18,Rok po wyborach trzeba powiedzieć jedno - nie ...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 16:12:19+00:00,[1846222583898784025],pl,1846222583898784000,"{'urls': [{'start': 100, 'end': 123, 'url': 'h...",False,Original,{'media_keys': ['13_1846222491456282626']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
2,"{'retweet_count': 4, 'reply_count': 3, 'like_c...",NaN,everyone,1182211615,NaN,1.8461614003280282e+18,"❌Mamy rok po wyborach, a Polska pogrąża się w ...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 12:09:12+00:00,[1846161400328028272],pl,1846161400328028160,"{'urls': [{'start': 278, 'end': 301, 'url': 'h...",False,Original,{'media_keys': ['3_1846148786910810112']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
3,"{'retweet_count': 6, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,NaN,1.8460918241017695e+18,Mija rok od wyborów parlamentarnych. W kampani...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 07:32:44+00:00,[1846091824101769490],pl,1846091824101769472,"{'urls': [{'start': 278, 'end': 301, 'url': 'h...",False,Original,{'media_keys': ['3_1846091818959597568']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
4,"{'retweet_count': 45, 'reply_count': 18, 'like...",NaN,everyone,1182211615,NaN,1.846075343188144e+18,#Idę11 🇵🇱 https://t.co/KiCe5ATOpX,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 06:27:14+00:00,[1846075343188144153],qme,1846075343188144128,"{'hashtags': [{'start': 0, 'end': 6, 'tag': 'I...",False,Original,{'media_keys': ['13_1846075276687478784']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja


In [105]:
# Get the value counts of the 'category' column
category_counts = df['category'].value_counts()

# Display the counts
print(category_counts)

# Get the number of unique categories
unique_category_count = category_counts.count()
print(f"Number of unique categories: {unique_category_count}")

category
Original    32794
Reply       10790
Quote        5478
Retweet      3480
Name: count, dtype: int64
Number of unique categories: 4


We delete retweets because they are wrongly provided by the X API. We want to analyze only original tweets, replies, and quotes.

In [106]:
df = df[df['category'] != 'Retweet']

In [107]:
df

,public_metrics,in_reply_to_user_id,reply_settings,author_id,context_annotations,id,text,edit_controls,referenced_tweets,created_at,edit_history_tweet_ids,lang,conversation_id,entities,possibly_sensitive,category,attachments,geo,username,party
0,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",375146901,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.8462772565091167e+18,"@donaldtusk Niezrealizowanie większości ze ""10...","{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'replied_to', 'id': '184609177626996...",2024-10-15 19:49:34+00:00,[1846277256509116623],pl,1846091776269963776,"{'mentions': [{'start': 0, 'end': 11, 'usernam...",False,Reply,NaN,NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
1,"{'retweet_count': 9, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.846222583898784e+18,Rok po wyborach trzeba powiedzieć jedno - nie ...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 16:12:19+00:00,[1846222583898784025],pl,1846222583898784000,"{'urls': [{'start': 100, 'end': 123, 'url': 'h...",False,Original,{'media_keys': ['13_1846222491456282626']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
2,"{'retweet_count': 4, 'reply_count': 3, 'like_c...",NaN,everyone,1182211615,NaN,1.8461614003280282e+18,"❌Mamy rok po wyborach, a Polska pogrąża się w ...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 12:09:12+00:00,[1846161400328028272],pl,1846161400328028160,"{'urls': [{'start': 278, 'end': 301, 'url': 'h...",False,Original,{'media_keys': ['3_1846148786910810112']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
3,"{'retweet_count': 6, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,NaN,1.8460918241017695e+18,Mija rok od wyborów parlamentarnych. W kampani...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 07:32:44+00:00,[1846091824101769490],pl,1846091824101769472,"{'urls': [{'start': 278, 'end': 301, 'url': 'h...",False,Original,{'media_keys': ['3_1846091818959597568']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
4,"{'retweet_count': 45, 'reply_count': 18, 'like...",NaN,everyone,1182211615,NaN,1.846075343188144e+18,#Idę11 🇵🇱 https://t.co/KiCe5ATOpX,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 06:27:14+00:00,[1846075343188144153],qme,1846075343188144128,"{'hashtags': [{'start': 0, 'end': 6, 'tag': 'I...",False,Original,{'media_keys': ['13_1846075276687478784']},NaN,bartlomiejpejo_2023-10-16_2024-10-15.json,Konfederacja
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52753,"{'retweet_count': 9, 'reply_count': 0, 'like_c...",NaN,everyone,961181894,NaN,1.7077195543553805e+18,"Studiujesz na kierunku lekarskim, pielęgniarst...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2023-09-29 11:30:44+00:00,[1707719554355380484],pl,1707719554355380480,"{'mentions': [{'start': 132, 'end': 142, 'user...",0,Original,{'media_keys': ['3_1707719240550166528']},NaN,StruzikAdam_2022-10-16_2023-10-15.json,PSL
52774,"{'retweet_count': 9, 'reply_count': 0, 'like_c...",NaN,everyone,961181894,NaN,1.7041203230234545e+18,Za nami posiedzenie @SejmikMaz. I kolejne wspa...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2023-09-19 13:08:40+00:00,[1704120323023454339],pl,1704120323023454464,"{'mentions': [{'start': 20, 'end': 30, 'userna...",0,Original,{'media_keys': ['3_1704118556785254400']},NaN,StruzikAdam_2022-10-16_2023-10-15.json,PSL
52777,"{'retweet_count': 8, 'reply_count': 0, 'like_c...",NaN,everyone,961181894,NaN,1.702668459576787e+18,Płockie Centrum Onkologii gotowe! Już na począ...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2023-09-15 12:59:29+00:00,[1702668459576787064],pl,1702668459576786944,"{'mentions': [{'start': 210, 'end': 220, 'user...",0,Original,{'media_keys': ['3_1702668263241498624']},NaN,StruzikAdam_2022-10-16_2023-10-15.json,PSL
52778,"{'re

In [108]:
# Update the 'username' column to keep only the string until '_2' -> split to date range

#df['username'] = df['username'].str.split('_2').str[0].copy()
df.loc[:, 'username'] = df['username'].str.split('_2').str[0]

In [109]:
category_summary = df['category'].value_counts()
print(category_summary)
total_tweets = category_summary.sum()
print(f"Total tweets: {total_tweets}")

category
Original    32794
Reply       10790
Quote        5478
Name: count, dtype: int64
Total tweets: 49062


In [110]:
# Ensure the created_at column is in datetime format

#df['created_at'] = pd.to_datetime(df['created_at'])
df.loc[:, 'created_at'] = pd.to_datetime(df['created_at'])

In [111]:
df.head()

,public_metrics,in_reply_to_user_id,reply_settings,author_id,context_annotations,id,text,edit_controls,referenced_tweets,created_at,edit_history_tweet_ids,lang,conversation_id,entities,possibly_sensitive,category,attachments,geo,username,party
0,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",375146901,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.8462772565091167e+18,"@donaldtusk Niezrealizowanie większości ze ""10...","{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'replied_to', 'id': '184609177626996...",2024-10-15 19:49:34+00:00,[1846277256509116623],pl,1846091776269963776,"{'mentions': [{'start': 0, 'end': 11, 'usernam...",False,Reply,NaN,NaN,bartlomiejpejo,Konfederacja
1,"{'retweet_count': 9, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.846222583898784e+18,Rok po wyborach trzeba powiedzieć jedno - nie ...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 16:12:19+00:00,[1846222583898784025],pl,1846222583898784000,"{'urls': [{'start': 100, 'end': 123, 'url': 'h...",False,Original,{'media_keys': ['13_1846222491456282626']},NaN,bartlomiejpejo,Konfederacja
2,"{'retweet_count': 4, 'reply_count': 3, 'like_c...",NaN,everyone,1182211615,NaN,1.8461614003280282e+18,"❌Mamy rok po wyborach, a Polska pogrąża się w ...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 12:09:12+00:00,[1846161400328028272],pl,1846161400328028160,"{'urls': [{'start': 278, 'end': 301, 'url': 'h...",False,Original,{'media_keys': ['3_1846148786910810112']},NaN,bartlomiejpejo,Konfederacja
3,"{'retweet_count': 6, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,NaN,1.8460918241017695e+18,Mija rok od wyborów parlamentarnych. W kampani...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 07:32:44+00:00,[1846091824101769490],pl,1846091824101769472,"{'urls': [{'start': 278, 'end': 301, 'url': 'h...",False,Original,{'media_keys': ['3_1846091818959597568']},NaN,bartlomiejpejo,Konfederacja
4,"{'retweet_count': 45, 'reply_count': 18, 'like...",NaN,everyone,1182211615,NaN,1.846075343188144e+18,#Idę11 🇵🇱 https://t.co/KiCe5ATOpX,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 06:27:14+00:00,[1846075343188144153],qme,1846075343188144128,"{'hashtags': [{'start': 0, 'end': 6, 'tag': 'I...",False,Original,{'media_keys': ['13_1846075276687478784']},NaN,bartlomiejpejo,Konfederacja


In [113]:
df.loc[1, 'text']

'Rok po wyborach trzeba powiedzieć jedno - nie na takie państwo Donald Tusk umawiał się z wyborcami! https://t.co/4Jh5Ni6sgr'

Emojis handler

In [114]:
def add_space_around_emojis(text):
    return ''.join(f' {char} ' if char in emoji.EMOJI_DATA or re.match(r'[\U0001F1E6-\U0001F1FF]', char) else char for char in text)

df['text'] = df['text'].apply(add_space_around_emojis)

def clean_text(text):
    mentions = re.findall(r'@\w+', text)
    text = re.sub(r'@\w+', '', text)
    links = re.findall(r'http\S+', text)
    text = re.sub(r'http\S+', '', text)
    hashtags = re.findall(r'#\w+', text)
    text = re.sub(r'(?<!\s)([\U0001F600-\U0001F64F])', r' \1', text)
    text = re.sub(r'([\U0001F600-\U0001F64F])(?!\s)', r'\1 ', text)
    return [text, mentions, links, hashtags]

df[['text_clean', 'mentions', 'links', 'hashtags']] = pd.DataFrame(df['text'].apply(clean_text).tolist(), index=df.index)

In [116]:
df.head()

,public_metrics,in_reply_to_user_id,reply_settings,author_id,context_annotations,id,text,edit_controls,referenced_tweets,created_at,...,possibly_sensitive,category,attachments,geo,username,party,text_clean,mentions,links,hashtags
0,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",375146901,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.8462772565091167e+18,"@donaldtusk Niezrealizowanie większości ze ""10...","{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'replied_to', 'id': '184609177626996...",2024-10-15 19:49:34+00:00,...,False,Reply,NaN,NaN,bartlomiejpejo,Konfederacja,"Niezrealizowanie większości ze ""100 konkretów...",[@donaldtusk],[],[]
1,"{'retweet_count': 9, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.846222583898784e+18,Rok po wyborach trzeba powiedzieć jedno - nie ...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 16:12:19+00:00,...,False,Original,{'media_keys': ['13_1846222491456282626']},NaN,bartlomiejpejo,Konfederacja,Rok po wyborach trzeba powiedzieć jedno - nie ...,[],[https://t.co/4Jh5Ni6sgr],[]
2,"{'retweet_count': 4, 'reply_count': 3, 'like_c...",NaN,everyone,1182211615,NaN,1.8461614003280282e+18,"❌ Mamy rok po wyborach, a Polska pogrąża się ...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 12:09:12+00:00,...,False,Original,{'media_keys': ['3_1846148786910810112']},NaN,bartlomiejpejo,Konfederacja,"❌ Mamy rok po wyborach, a Polska pogrąża się ...",[],"[https://t.co/zFk5QLd1em, https://t.co/bRV4y07...",[]
3,"{'retweet_count': 6, 'reply_count': 2, 'like_c...",NaN,everyone,1182211615,NaN,1.8460918241017695e+18,Mija rok od wyborów parlamentarnych. W kampani...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 07:32:44+00:00,...,False,Original,{'media_keys': ['3_1846091818959597568']},NaN,bartlomiejpejo,Konfederacja,Mija rok od wyborów parlamentarnych. W kampani...,[],"[https://t.co/rtVu3Bh43G, https://t.co/8Q3LME6...",[]
4,"{'retweet_count': 45, 'reply_count': 18, 'like...",NaN,everyone,1182211615,NaN,1.846075343188144e+18,#Idę11 🇵 🇱 https://t.co/KiCe5ATOpX,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 06:27:14+00:00,...,False,Original,{'media_keys': ['13_1846075276687478784']},NaN,bartlomiejpejo,Konfederacja,#Idę11 🇵 🇱,[],[https://t.co/KiCe5ATOpX],[#Idę11]


In [117]:
df.drop(columns=['entities'], inplace=True)

In [ ]:
# Some additional numerical data from tweets is extracted and added to the dataframe as new variables, then the original column is dropped
df['retweet_count'] = df['public_metrics'].apply(lambda x: x['retweet_count'])
df['reply_count'] = df['public_metrics'].apply(lambda x: x['reply_count'])
df['like_count'] = df['public_metrics'].apply(lambda x: x['like_count'])
df['quote_count'] = df['public_metrics'].apply(lambda x: x['quote_count'])
df['impression_count'] = df['public_metrics'].apply(lambda x: x['impression_count'])

df.drop(columns=['public_metrics'], inplace=True)

In [119]:
df

,in_reply_to_user_id,reply_settings,author_id,context_annotations,id,text,edit_controls,referenced_tweets,created_at,edit_history_tweet_ids,...,party,text_clean,mentions,links,hashtags,retweet_count,reply_count,like_count,quote_count,impression_count
0,375146901,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.8462772565091167e+18,"@donaldtusk Niezrealizowanie większości ze ""10...","{'edits_remaining': 5, 'is_edit_eligible': Fal...","[{'type': 'replied_to', 'id': '184609177626996...",2024-10-15 19:49:34+00:00,[1846277256509116623],...,Konfederacja,"Niezrealizowanie większości ze ""100 konkretów...",[@donaldtusk],[],[],3,1,33,0,1555
1,NaN,everyone,1182211615,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1.846222583898784e+18,Rok po wyborach trzeba powiedzieć jedno - nie ...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 16:12:19+00:00,[1846222583898784025],...,Konfederacja,Rok po wyborach trzeba powiedzieć jedno - nie ...,[],[https://t.co/4Jh5Ni6sgr],[],9,2,72,0,3031
2,NaN,everyone,1182211615,NaN,1.8461614003280282e+18,"❌ Mamy rok po wyborach, a Polska pogrąża się ...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 12:09:12+00:00,[1846161400328028272],...,Konfederacja,"❌ Mamy rok po wyborach, a Polska pogrąża się ...",[],"[https://t.co/zFk5QLd1em, https://t.co/bRV4y07...",[],4,3,33,2,8636
3,NaN,everyone,1182211615,NaN,1.8460918241017695e+18,Mija rok od wyborów parlamentarnych. W kampani...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 07:32:44+00:00,[1846091824101769490],...,Konfederacja,Mija rok od wyborów parlamentarnych. W kampani...,[],"[https://t.co/rtVu3Bh43G, https://t.co/8Q3LME6...",[],6,2,38,0,2441
4,NaN,everyone,1182211615,NaN,1.846075343188144e+18,#Idę11 🇵 🇱 https://t.co/KiCe5ATOpX,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2024-10-15 06:27:14+00:00,[1846075343188144153],...,Konfederacja,#Idę11 🇵 🇱,[],[https://t.co/KiCe5ATOpX],[#Idę11],45,18,616,2,8634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52753,NaN,everyone,961181894,NaN,1.7077195543553805e+18,"Studiujesz na kierunku lekarskim, pielęgniarst...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2023-09-29 11:30:44+00:00,[1707719554355380484],...,PSL,"Studiujesz na kierunku lekarskim, pielęgniarst...",[@SejmikMaz],[https://t.co/6zats7JXbY],[],9,0,6,0,2154
52774,NaN,everyone,961181894,NaN,1.7041203230234545e+18,Za nami posiedzenie @SejmikMaz. I kolejne wspa...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2023-09-19 13:08:40+00:00,[1704120323023454339],...,PSL,Za nami posiedzenie . I kolejne wsparcie dla m...,[@SejmikMaz],[https://t.co/A7EG9Jzuv1],[#OSP],9,0,15,0,649
52777,NaN,everyone,961181894,NaN,1.702668459576787e+18,Płockie Centrum Onkologii gotowe! Już na począ...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2023-09-15 12:59:29+00:00,[1702668459576787064],...,PSL,Płockie Centrum Onkologii gotowe! Już na począ...,[@SejmikMaz],[https://t.co/OALgj7gqxE],[],8,0,16,0,581
52778,NaN,everyone,961181894,NaN,1.7019609093698685e+18,To jedna z największych inwestycji drogowych @...,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,2023-09-13 14:07:56+00:00,[1701960909369868437],...,PSL,To jedna z największych inwestycji drogowych ...,[@SejmikMaz],[https://t.co/9jWRcVZHXk],[#634],8,0,13,0,621


In [120]:
df.dtypes

in_reply_to_user_id                   float64
reply_settings                         object
author_id                             float64
context_annotations                    object
id                                     object
text                                   object
edit_controls                          object
referenced_tweets                      object
created_at                datetime64[ns, UTC]
edit_history_tweet_ids                 object
lang                                   object
conversation_id                       float64
possibly_sensitive                     object
category                               object
attachments                            object
geo                                    object
username                               object
party                                  object
text_clean                             object
mentions                               object
links                                  object
hashtags                          

In [121]:
# Check for duplicate columns and remove them
if df.columns.duplicated().any():
    print("Duplicate columns found! Removing them...")
    df_no_duplicates = df.loc[:, ~df.columns.duplicated()]  # Keep the first occurrence of each column
else:
    df_no_duplicates = df.copy()

#  Convert 'id' column to string (if needed)
df_no_duplicates['id'] = df_no_duplicates['id'].astype(str)

# Check for missing or empty values in 'text' and 'text_clean'
empty_text = df_no_duplicates[df_no_duplicates['text'].isna() | (df_no_duplicates['text'].astype(str).str.strip() == '')]
empty_text_clean = df_no_duplicates[df_no_duplicates['text_clean'].isna() | (df_no_duplicates['text_clean'].astype(str).str.strip() == '')]

print(f"Rows where 'text' is empty or null: {empty_text.shape[0]}")
print(empty_text[['id', 'text', 'text_clean']].head())

print(f"\nRows where 'text_clean' is empty or null: {empty_text_clean.shape[0]}")
empty_text_clean[['id', 'text', 'text_clean']].head()


Rows where 'text' is empty or null: 0
Empty DataFrame
Columns: [id, text, text_clean]
Index: []

Rows where 'text_clean' is empty or null: 731


,id,text,text_clean
16,1.8447112765779645e+18,@Nowa_Nadzieja_ @KONFEDERACJA_,
92,1.8386217610902856e+18,@KONFEDERACJA_ @Nowa_Nadzieja_,
254,1.8215209926293056e+18,https://t.co/H9BQbYjylo,
595,1.7682138922729597e+18,@MPerspektywa @AdamAbramowicz1 https://t.co/bz...,
786,1.7333981159174026e+18,https://t.co/uXKjbwD1DQ,


In [122]:
false_count = (df['text_clean'].str.strip().astype(bool))
print(f"Number of False values: {false_count}")

Number of False values: 0        True
1        True
2        True
3        True
4        True
         ... 
52753    True
52774    True
52777    True
52778    True
52779    True
Name: text_clean, Length: 49062, dtype: bool


In [123]:
#Delete empty tweets
df = df[false_count]

Saving data used for translation 

In [73]:
df_clean_text = df[['id', 'text', 'text_clean']]

df_clean_text.to_csv('../data/02.processed/data_for_translation.csv', index=False)
df.to_csv('../data/02.processed/whole_dataset_for_translation.csv', index=False)

Reading translated dataset 

In [ ]:
df_en_text = pd.read_parquet('../data/02.processed/df_translated.parquet')

Merging translated tweets with original one

In [63]:
# Step 1: Make sure IDs are strings
df['id'] = df['id'].astype(str)
df_en_text['id'] = df_en_text['id'].astype(str)

df_en_combined = pd.concat([df_en_text], ignore_index=True)

# Step 3: Drop duplicates by 'id' to keep only the latest 
df_en_combined = df_en_combined.drop_duplicates(subset='id', keep='last')

# Step 4: Merge back into the full dataset to get a unified view
df_merged = df.merge(df_en_text[['id', 'text_clean_en']], on='id')

print(f"Total rows after merge: {len(df_merged)} ")


Total rows after merge: 48331 


In [64]:
# Find the IDs present in df but not in df_merged
missing_ids = df[~df['id'].isin(df_merged['id'])]['id']

# Display the missing rows
missing_rows = df[df['id'].isin(missing_ids)]
print(missing_rows)

Empty DataFrame
Columns: [reply_settings, created_at, attachments, edit_controls, author_id, edit_history_tweet_ids, lang, possibly_sensitive, id, conversation_id, text, category, context_annotations, in_reply_to_user_id, referenced_tweets, username, party, geo, text_clean, mentions, links, hashtags, retweet_count, reply_count, like_count, quote_count, impression_count]
Index: []

[0 rows x 27 columns]


In [65]:
df_merged

,reply_settings,created_at,attachments,edit_controls,author_id,edit_history_tweet_ids,lang,possibly_sensitive,id,conversation_id,...,text_clean,mentions,links,hashtags,retweet_count,reply_count,like_count,quote_count,impression_count,text_clean_en
0,everyone,2024-10-15 07:13:34+00:00,"{'media_keys': ['3_1846083966849159168', '3_18...","{'edits_remaining': 4, 'is_edit_eligible': Tru...",1284852220593414144,"[1846086770229694583, 1846086999964283214]",pl,False,1846086999964283136,1846086999964283136,...,❌ Rząd polski zamierza budować w Polsce 49 C...,[],"[https://t.co/gL3O8F0ITB, https://t.co/cay37TX...",[],230,61,644,7,11648,❌ The Polish government intends to build 49 F...
1,everyone,2024-10-14 08:46:51+00:00,NaN,"{'edits_remaining': 4, 'is_edit_eligible': Tru...",1284852220593414144,"[1845747336862961872, 1845748090461966651]",pl,False,1845748090461966592,1845748090461966592,...,❌ Szambo wybija i robi się coraz ciekawiej. ...,[],[https://t.co/OZyTSwcMAb],[],1301,169,3845,57,146584,❌ The cesspool is breaking out and it's getti...
2,everyone,2024-10-13 07:30:58+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1284852220593414144,[1845366606823657982],pl,False,1845366606823657984,1845366606823657984,...,"❌ NIE ROZUMIEM, JAK MOŻNA KRZYWDZIĆ W TEN SP...",[@donaldtusk],[https://t.co/rIGkIpR8sw],[#RadaKrajowaKO],682,145,2061,28,100757,❌ I DON'T UNDERSTAND HOW YOU CAN HURT YOUR OW...
3,everyone,2024-10-12 07:38:50+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1284852220593414144,[1845006197847359885],pl,False,1845006197847360000,1845006197847360000,...,🆘 Firma farmaceutyczna GSK zapłaci ponad 2 ...,[],[https://t.co/cwusG1221F],[],271,56,989,8,30769,🆘 The pharmaceutical company GSK will pay ove...
4,everyone,2024-10-11 06:56:29+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1284852220593414144,[1844633149784891665],pl,False,1844633149784891648,1844633149784891648,...,❌ O CO TUTAJ CHODZI? W październiku 2024 r. ...,"[@MZ_GOV_PL, @Leszczyna, @NFZ_GOV_PL]",[https://t.co/RkylDcUHbo],[],214,56,678,9,17432,"❌ WHAT IS GOING ON HERE? In October 2024, her..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48326,everyone,2022-11-04 17:02:07+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1119834276,[1588577342594482176],pl,0,1588577342594482176,1588577342594482176,...,Rozporządzenie o zakazie rejestracji nowych sa...,[@EPPGroup],[https://t.co/zjdg6y5yH8],[],3,0,3,1,0,Regulation banning the registration of new car...
48327,everyone,2022-11-04 07:02:07+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1119834276,[1588426347646533634],pl,0,1588426347646533632,1588426347646533632,...,Wojna gazowa Putina. Europa przełamała szantaż...,[@Money_pl],[https://t.co/UP6coX7Ukj],[],1,0,2,1,0,Putin's gas war. Europe has broken the blackma...
48328,everyone,2022-10-29 12:23:28+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1119834276,[1586332889444237312],pl,0,1586332889444237312,1586332889444237312,...,"Naukowcy wiedzą jak sprawić, by OZE nie zależa...",[],[https://t.co/rS6OtqX4wc],[],3,0,2,1,0,Scientists know how to ensure that renewable e...
48329,everyone,2022-10-21 07:33:09+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1119834276,[1583360725958828034],pl,0,1583360725958828032,1583360725958828032,...,Chaos na rynku energii. Zrywane umowy i zamroż...,[@BOksinska],[https://t.co/XMl5UjJLwb],[],4,0,2,1,0,Chaos on the energy market. Terminated contrac...


In [ ]:
# Check how many rows still have missing or empty translations
missing_translation_mask = df_merged['text_clean_en'].isna() | (df_merged['text_clean_en'].str.strip() == '')

# Show some of them
df_missing_translation = df_merged[missing_translation_mask]
print(f"Rows without translation: {df_missing_translation.shape[0]}")
display(df_missing_translation[['id', 'text', 'text_clean', 'text_clean_en']].head())

# Remove them
df_clean_translated = df_merged[~missing_translation_mask].copy()

In [ ]:
df_clean_translated.to_csv('../data/02.processed/df_clean_translated_further_analalysis.csv', index=False)

In [71]:
def count_emojis(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & pictographs
        "\U0001F680-\U0001F6FF"  # Transport & map symbols
        "\U0001F700-\U0001F77F"  # Alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric shapes
        "\U0001F800-\U0001F8FF"  # Supplemental arrows
        "\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
        "\U0001FA00-\U0001FA6F"  # Chess symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed characters
        "]+",
        flags=re.UNICODE,
    )
    return len(emoji_pattern.findall(text))


In [72]:
# Demojize text columns
df_clean_translated['text_clean_en_demojized'] = df_clean_translated['text_clean_en'].apply(
    lambda x: emoji.demojize(str(x)) if pd.notnull(x) else x
)
df_clean_translated['text_clean_demojized'] = df_clean_translated['text_clean'].apply(
    lambda x: emoji.demojize(str(x)) if pd.notnull(x) else x
)

# Count emojis in original text columns
df_clean_translated['emoji_count_en'] = df_clean_translated['text_clean_en'].apply(
    lambda x: count_emojis(str(x)) if pd.notnull(x) else 0
)
df_clean_translated['emoji_count'] = df_clean_translated['text_clean'].apply(
    lambda x: count_emojis(str(x)) if pd.notnull(x) else 0
)


In [74]:
df_clean_translated[['text_clean_en', 'text_clean_en_demojized', 'emoji_count_en', 'text_clean', 'text_clean_demojized', 'emoji_count']].head()


,text_clean_en,text_clean_en_demojized,emoji_count_en,text_clean,text_clean_demojized,emoji_count
0,❌ The Polish government intends to build 49 F...,:cross_mark: The Polish government intends to...,2,❌ Rząd polski zamierza budować w Polsce 49 C...,:cross_mark: Rząd polski zamierza budować w ...,3
1,❌ The cesspool is breaking out and it's getti...,:cross_mark: The cesspool is breaking out and...,1,❌ Szambo wybija i robi się coraz ciekawiej. ...,:cross_mark: Szambo wybija i robi się coraz ...,1
2,❌ I DON'T UNDERSTAND HOW YOU CAN HURT YOUR OW...,:cross_mark: I DON'T UNDERSTAND HOW YOU CAN H...,1,"❌ NIE ROZUMIEM, JAK MOŻNA KRZYWDZIĆ W TEN SP...",":cross_mark: NIE ROZUMIEM, JAK MOŻNA KRZYWDZ...",1
3,🆘 The pharmaceutical company GSK will pay ove...,:SOS_button: The pharmaceutical company GSK w...,2,🆘 Firma farmaceutyczna GSK zapłaci ponad 2 ...,:SOS_button: Firma farmaceutyczna GSK zapła...,3
4,"❌ WHAT IS GOING ON HERE? In October 2024, her...",:cross_mark: WHAT IS GOING ON HERE? In Octobe...,2,❌ O CO TUTAJ CHODZI? W październiku 2024 r. ...,:cross_mark: O CO TUTAJ CHODZI? W październi...,3


In [75]:
# Filter rows
rows_with_emojis_in_text_clean_only = df_clean_translated[
    (df_clean_translated['emoji_count'] > 0) & (df_clean_translated['emoji_count_en'] == 0)
]

# Display the number of such rows
print(f"Number of rows with emojis in 'text_clean' but not in 'text_clean_en': {len(rows_with_emojis_in_text_clean_only)}")

# Display the affected rows
rows_with_emojis_in_text_clean_only[['text_clean', 'text_clean_en']]


Number of rows with emojis in 'text_clean' but not in 'text_clean_en': 295


,text_clean,text_clean_en
472,Recepta na problemy na polskiej granicy jest b...,The solution to problems at the Polish border ...
536,Unia Europejska mówi Polakom: wasze domy i mie...,The European Union tells Poles: your houses an...
633,Unia Europejska planuje wprowadzać kolejne ogr...,The European Union plans to introduce further ...
651,Poseł Adam Gomoła z Polski 2050 Szymona Hołown...,MP Adam Gomoła from Szymon Hołownia's Poland 2...
686,"Premier Tusk oświadczył, że zamierza wprowadzi...",Prime Minister Tusk stated that he intends to ...
...,...,...
45637,W niedzielę będę gościem . Serdecznie zachęcam...,I will be a guest on Sunday. I encourage you t...
45654,"I 10 razy tyle, co na mieszkalnictwo 🙂",And 10 times as much as for housing :)
46684,Intensywny tydzień misji Komisji Budżetu PE w ...,An intense week of mission of the EP Budget Co...
46926,Może być 🙂 Dziękuję i ściskam dłoń,Maybe :) Thank you and I shake your hand


In [77]:
df_clean_translated['possibly_sensitive'] = df_clean_translated['possibly_sensitive'].astype(bool)

In [78]:
username_to_realname = {
    'bartlomiejpejo': 'Bartłomiej Pejo',
    'GrzegorzBraun_': 'Grzegorz Braun',
    'Iwaszkiewicz_RJ': 'Robert Iwaszkiewicz',
    'KonradBerkowicz': 'Konrad Berkowicz',
    'MarSypniewski': 'Marcin Sypniewski',
    'MichalWawer': 'Michał Wawer',
    'placzekgrzegorz': 'Grzegorz Płaczek',
    'SlawomirMentzen': 'Sławomir Mentzen',
    'TudujKrzysztof': 'Krzysztof Tuduj',
    'Wlodek_Skalik': 'Włodzimierz Skalik',
    'WTumanowicz': 'Witold Tumanowicz',
    'AndrzejSzejna': 'Andrzej Szejna',
    'AnitaKDZG': 'Anita Kucharska-Dziedzic',
    'JoankaSW': 'Joanna Scheuring-Wielgus',
    'KGawkowski': 'Krzysztof Gawkowski',
    'K_Smiszek': 'Krzysztof Śmiszek',
    'MarcinKulasek': 'Marcin Kulasek',
    'MoskwaWodnicka': 'Małgorzata Moskwa-Wodnicka',
    'PaulinaPW2024': 'Paulina Piechna-Więckiewicz',
    'poselTTrela': 'Tomasz Trela',
    'RobertBiedron': 'Robert Biedroń',
    'WandaNowicka': 'Wanda Nowicka',
    'wieczorekdarek': 'Dariusz Wieczorek',
    'wlodekczarzasty': 'Włodzimierz Czarzasty',
    'Arek_Iwaniak': 'Arkadiusz Iwaniak',
    'B_Maciejewska': 'Beata Maciejewska',
    'BeataSzydlo': 'Beata Szydło',
    'elzbietawitek': 'Elżbieta Witek',
    'Kaminski_M_': 'Mariusz Kamiński',
    'Kowalczyk_H': 'Henryk Kowalczyk',
    'Macierewicz_A': 'Antoni Macierewicz',
    'mblaszczak': 'Mariusz Błaszczak',
    'MorawieckiM': 'Mateusz Morawiecki',
    'mwojcik_': 'Michał Wójcik',
    'PatrykJaki': 'Patryk Jaki',
    'bbudka': 'Borys Budka',
    'CTomczyk': 'Cezary Tomczyk',
    'donaldtusk': 'Donald Tusk',
    'DorotaNiedziela': 'Dorota Niedziela',
    'EwaKopacz': 'Ewa Kopacz',
    'JanGrabiec': 'Jan Grabiec',
    'Konwinski_PO': 'Zbigniew Konwiński',
    'Leszczyna': 'Izabela Leszczyna',
    'MKierwinski': 'Marcin Kierwiński',
    'M_K_Blonska': 'Małgorzata Kidawa-Błońska',
    'OklaDrewnowicz': 'Agnieszka Okła-Drewnowicz',
    'trzaskowski_': 'Rafał Trzaskowski',
    'TomaszSiemoniak': 'Tomasz Siemoniak',
    'AgaBaranowskaPL': 'Agnieszka Baranowska',
    'aga_buczynska': 'Agnieszka Buczyńska',
    'hennigkloska': 'Paulina Hennig-Kloska',
    'joannamucha': 'Joanna Mucha',
    'Kpelczynska': 'Katarzyna Pełczyńska-Nałęcz',
    'LukaszOsmalak': 'Łukasz Osmalak',
    'SlizPawel': 'Paweł Śliz',
    'szymon_holownia': 'Szymon Hołownia',
    'ZalewskiPawel': 'Paweł Zalewski',
    'ZywnoMaciej': 'Maciej Żywno',
    'JKozlowskiEu': 'Jacek Kozłowski',
    'michalkobosko': 'Michał Kobosko',
    'DariuszKlimczak': 'Dariusz Klimczak',
    'GrzybAndrzej': 'Andrzej Grzyb',
    'Hetman_K': 'Krzysztof Hetman',
    'JarubasAdam': 'Adam Jarubas',
    'KosiniakKamysz': 'Władysław Kosiniak-Kamysz',
    'Paslawska': 'Urszula Pasławska',
    'PZgorzelskiP': 'Piotr Zgorzelski',
    'StefanKrajewski': 'Stefan Krajewski',
    'StruzikAdam': 'Adam Struzik'
}

# Add the 'name' column to the dataframe
df_clean_translated['name'] = df_clean_translated['username'].map(username_to_realname)

In [79]:
# Delete next line sign from the 'text_clean_en' column
df_clean_translated['text_clean_en'] = df_clean_translated['text_clean_en'].str.replace('\n', ' ')

In [ ]:
# Save the DataFrame to a Parquet file
df_clean_translated.to_parquet('../data/03.cleaned/df_combined.parquet', index=False)

In [82]:
df_clean_translated

,reply_settings,created_at,attachments,edit_controls,author_id,edit_history_tweet_ids,lang,possibly_sensitive,id,conversation_id,...,reply_count,like_count,quote_count,impression_count,text_clean_en,text_clean_en_demojized,text_clean_demojized,emoji_count_en,emoji_count,name
0,everyone,2024-10-15 07:13:34+00:00,"{'media_keys': ['3_1846083966849159168', '3_18...","{'edits_remaining': 4, 'is_edit_eligible': Tru...",1284852220593414144,"[1846086770229694583, 1846086999964283214]",pl,False,1846086999964283136,1846086999964283136,...,61,644,7,11648,❌ The Polish government intends to build 49 F...,:cross_mark: The Polish government intends to...,:cross_mark: Rząd polski zamierza budować w ...,2,3,Grzegorz Płaczek
1,everyone,2024-10-14 08:46:51+00:00,NaN,"{'edits_remaining': 4, 'is_edit_eligible': Tru...",1284852220593414144,"[1845747336862961872, 1845748090461966651]",pl,False,1845748090461966592,1845748090461966592,...,169,3845,57,146584,❌ The cesspool is breaking out and it's getti...,:cross_mark: The cesspool is breaking out and...,:cross_mark: Szambo wybija i robi się coraz ...,1,1,Grzegorz Płaczek
2,everyone,2024-10-13 07:30:58+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1284852220593414144,[1845366606823657982],pl,False,1845366606823657984,1845366606823657984,...,145,2061,28,100757,❌ I DON'T UNDERSTAND HOW YOU CAN HURT YOUR OW...,:cross_mark: I DON'T UNDERSTAND HOW YOU CAN H...,":cross_mark: NIE ROZUMIEM, JAK MOŻNA KRZYWDZ...",1,1,Grzegorz Płaczek
3,everyone,2024-10-12 07:38:50+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1284852220593414144,[1845006197847359885],pl,False,1845006197847360000,1845006197847360000,...,56,989,8,30769,🆘 The pharmaceutical company GSK will pay ove...,:SOS_button: The pharmaceutical company GSK w...,:SOS_button: Firma farmaceutyczna GSK zapła...,2,3,Grzegorz Płaczek
4,everyone,2024-10-11 06:56:29+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1284852220593414144,[1844633149784891665],pl,False,1844633149784891648,1844633149784891648,...,56,678,9,17432,"❌ WHAT IS GOING ON HERE? In October 2024, her...",:cross_mark: WHAT IS GOING ON HERE? In Octobe...,:cross_mark: O CO TUTAJ CHODZI? W październi...,2,3,Grzegorz Płaczek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48326,everyone,2022-11-04 17:02:07+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1119834276,[1588577342594482176],pl,False,1588577342594482176,1588577342594482176,...,0,3,1,0,Regulation banning the registration of new car...,Regulation banning the registration of new car...,Rozporządzenie o zakazie rejestracji nowych sa...,0,0,Andrzej Grzyb
48327,everyone,2022-11-04 07:02:07+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1119834276,[1588426347646533634],pl,False,1588426347646533632,1588426347646533632,...,0,2,1,0,Putin's gas war. Europe has broken the blackma...,Putin's gas war. Europe has broken the blackma...,Wojna gazowa Putina. Europa przełamała szantaż...,0,0,Andrzej Grzyb
48328,everyone,2022-10-29 12:23:28+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1119834276,[1586332889444237312],pl,False,1586332889444237312,1586332889444237312,...,0,2,1,0,Scientists know how to ensure that renewable e...,Scientists know how to ensure that renewable e...,"Naukowcy wiedzą jak sprawić, by OZE nie zależa...",0,0,Andrzej Grzyb
48329,everyone,2022-10-21 07:33:09+00:00,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1119834276,[1583360725958828034],pl,False,1583360725958828032,1583360725958828032,...,0,2,1,0,Chaos on the energy market. Terminated contrac...,Chaos on the energy market. Terminated contrac...,Chaos na rynku energii. Zrywane umowy i zamroż...,0,0,Andrzej Grzyb
